In [1]:
import numpy as np 
import pandas as pd
import os
import psycopg2
import geopandas as gpd

import importlib
from Secrets import secrets
# importlib.reload(secrets)

import matplotlib.pyplot as plt
import seaborn as sns

### Analysis
 
 - Crime rate over time by category
 - - Areas with most crime rate by type
 - - Crime rate by areas and over time
 
 - Best areas for grocery stores
 - Best areas for going out 
 - Best areas for school catchment
 
 - Relationship between avg price, gdp, hpi
 - Areas that under and over perform hpi index
 
 - estimate premium paid per sqft for good schools, stores vibrant areas
 - estimate drivers for price